In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
import spm1d.stats.c
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors.v2021 import register_cmaps
import spm1d
from stressgait_analysis.dataset import StressGaitDataset



%matplotlib widget
%load_ext autoreload
%autoreload 2
from stressgait_analysis.gait_helper_functions import (compute_HS)
import numpy as np

In [ ]:
plt.close("all")
register_cmaps()
palette = sns.color_palette("faculties_light")
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"


controls_color = sns.color_palette("wiso")[1]
omc_color = sns.color_palette("med_dark")[1]

stressgait = {
    "OMC": omc_color,
    "Control": controls_color
}

In [ ]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])

In [ ]:
dataset = StressGaitDataset(base_path, coarse_condition=True, specify_bouts=True, specify_speed=True)

In [ ]:
kinematics = dataset.kinematics.droplevel(level="bout")
cycle_averaged_q = kinematics.groupby(["participant", "condition", "speed", "percentage_of_stride"]).mean()
cycle_averaged_q#.head()

split into omc / control as well as fast / slow

In [ ]:
omc_kinematics_slow = cycle_averaged_q.xs(("OMC", "slow"), level=("condition", "speed"))
omc_kinematics_fast = cycle_averaged_q.xs(("OMC", "fast"), level=("condition", "speed"))

control_kinematics_slow = cycle_averaged_q.xs(("Control", "slow"), level=("condition", "speed"))
control_kinematics_fast = cycle_averaged_q.xs(("Control", "fast"), level=("condition", "speed"))

for spm, we need data in the format J x Q, where J are the participants per group and Q are the nodes

In [ ]:
omc_slow_hip = omc_kinematics_slow.reset_index().pivot(index="participant", columns="percentage_of_stride", values="hip_flexion")
omc_slow_knee = omc_kinematics_slow.reset_index().pivot(index="participant", columns="percentage_of_stride", values="knee_flexion")

control_slow_hip = control_kinematics_slow.reset_index().pivot(index="participant", columns="percentage_of_stride", values="hip_flexion")
control_slow_knee = control_kinematics_slow.reset_index().pivot(index="participant", columns="percentage_of_stride", values="knee_flexion")

omc_fast_hip = omc_kinematics_fast.reset_index().pivot(index="participant", columns="percentage_of_stride", values="hip_flexion")
omc_fast_knee = omc_kinematics_fast.reset_index().pivot(index="participant", columns="percentage_of_stride", values="knee_flexion")

control_fast_hip = control_kinematics_fast.reset_index().pivot(index="participant", columns="percentage_of_stride", values="hip_flexion")
control_fast_knee = control_kinematics_fast.reset_index().pivot(index="participant", columns="percentage_of_stride", values="knee_flexion")


In [ ]:
def plot_result(YA, YB, ti, ylabel=None):
    sns.set_style("white")
### plot mean and SD:
    fig = plt.figure( figsize=(8, 3.5) )
    ax     = plt.axes( (0.1, 0.15, 0.35, 0.8) )
    spm1d.plot.plot_mean_sd(YA, linecolor=stressgait["OMC"],label="OMC", facecolor=stressgait["OMC"], ax=ax)
    spm1d.plot.plot_mean_sd(YB,  linecolor=stressgait["Control"], label= "Control", facecolor=stressgait["Control"], ax=ax)

    ax.set_xlabel('% of stride', fontsize=16)
    ax.tick_params(axis='both',labelsize=12)
    ax.set_ylabel(ylabel, fontsize=16)
    ax.tick_params(axis='both', colors='black', labelsize=12, bottom=True, left=True, labelbottom=True)
    plt.legend()
    ### plot SPM results:
    ax= plt.axes((0.55,0.15,0.35,0.8))

    ti.plot(facecolor="red")

    ax.set_ylabel( "SPM{t}",fontsize=16)
   # ti.plot_threshold_label(fontsize=8)
    alpha = 0.0125
    tstar = ti.zstar  # or use the threshold value directly

# Optional: plot the threshold line manually if not already present
#    plt.axhline(y=tstar, linestyle='--', color='k')

# Add your custom label
    plt.text(
    x=60,       # e.g. 5 or plt.xlim()[0] + some_offset
    y=tstar + 0.1,           # position slightly above the line
    s=f"α = {alpha:.4f}, t* = {tstar:.3f}",
    fontsize=12,
    color='red',
    ha='center'
)
    ti.plot_p_values(size=14, offsets=[(30,0.5)])
    ax.set_xlabel('% of stride', fontsize=16)
    ax.tick_params(axis='both', colors='black', labelsize=12, left=True, bottom=True, labelbottom=True)

    return fig

## SPM for the hip

### slow

In [ ]:
t = spm1d.stats.ttest2(np.array(omc_slow_hip), np.array(control_slow_hip))
ti = t.inference(alpha=0.05/4)
figure = plot_result(omc_slow_hip, control_slow_hip, ti, "Hip flexion [°]")
plt.savefig("../../exports/plots/spm_hip_slow.pdf")

### fast

In [ ]:
t = spm1d.stats.ttest2(np.array(omc_fast_hip), np.array(control_fast_hip))
ti = t.inference(alpha=0.05/4)
figure = plot_result(omc_fast_hip, control_fast_hip, ti)
plt.savefig("../../exports/plots/spm_hip_fast.pdf")

## knee

### slow

In [ ]:
t = spm1d.stats.ttest2(np.array(omc_slow_knee), np.array(control_slow_knee))
ti = t.inference(alpha=0.05/4)
figure = plot_result(omc_slow_knee, control_slow_knee, ti, "Knee flexion [°]")
plt.savefig("../../exports/plots/spm_knee_slow.pdf")

In [ ]:
ti.clusters

### fast

In [ ]:
t = spm1d.stats.ttest2(np.array(omc_fast_knee), np.array(control_fast_knee))
ti = t.inference(alpha=0.05/4)
figure = plot_result(omc_fast_knee, control_fast_knee, ti)
plt.savefig("../../exports/plots/spm_knee_fast.pdf")

In [ ]:
cycle_averaged_q = kinematics.groupby(["participant", "condition", "speed", "percentage_of_stride"]).mean()
cycle_averaged_q.head()
c = cycle_averaged_q.reset_index()

In [ ]:
responder_list = ["VP_02", "VP_05", "VP_06", "VP_15", "VP_16", "VP_17", "VP_24", "VP_35", "VP_36", "VP_43", "VP_45", "VP47"]
cycle_averaged_q["responder"] = [1 if r[1].participant in responder_list else 0 for r in c.iterrows()]
cycle_averaged_q.set_index("responder", append=True, inplace=True)

In [ ]:
cycle_averaged_q